In [1]:
!pip install redis

TODO: publish/subscribe function within redis?

In [2]:
from flask import Flask, request, jsonify, redirect, render_template
import redis
import pandas as pd
import datetime as dt

In [3]:
# Connect to redis
redis_db = redis.Redis(host='localhost', port=6379)

In [4]:
## Create Flask API
api = Flask(__name__)

#### CREATE STOCKMENTION

Key = ticker:date \
Value(Hash) = (price, mention) \

In [5]:
# Delete all in redis
#redis_db.flushall()
#len(redis_db.keys())
#insert_ticker_values('GME','2021-05-24','180.00999450683594','11')


In [6]:
@api.route("/ticker/<string:ticker>/<string:date>/<int:price>/<int:mentions>", methods=["POST"])
def insert_ticker_values(ticker, date, price, mentions):
    #expire_date = date.datetime.datetime.strptime(start_date, "%m/%d/%y") + datetime.timedelta(days=7)
    #time_to_expire_s = str(dt.datetime.strptime(expire_date, "%Y-%m-%d").timestamp())
    
    values = {"price":price, "mentions":mentions}
    try:
        transaction = redis_db.pipeline(transaction=True)
        #transaction.sadd("ticker", date_utc)
        transaction.hmset(ticker +':'+ date, values)#, ex=time_to_expire_s      
        transaction.execute()
        return(f"values for "+ticker+" were inserted")
    except Exception as e:
        print(e)

## GET STOCK OVERVIEW

In [7]:
@api.route("/weekly/<string:ticker>", methods=["GET"])
def get_weekly(ticker): 
    #utc_from = dt.datetime.strptime(date_from, "%Y-%m-%d").timestamp()
    #utc_to = dt.datetime.strptime(date_to, "%Y-%m-%d").timestamp()
    #if redis_db.exists(ticker):
    weekly_overview = pd.DataFrame()
    for key in redis_db.keys("GME:*"):
        key = key.decode('utf-8')
        date = key[4:]
        values = redis_db.hmget(key, 'price', 'mentions')
        
        dayly_overview = {
            'Date': date,
            'Price': values[0].decode('utf-8'),
            'Mentions': values[1].decode('utf-8')
        }
        weekly_overview = weekly_overview.append(dayly_overview, ignore_index=True)
        weekly_overview.sort_values('Date')
        
    return weekly_overview.to_json(orient='records')

In [8]:
@api.route("/span/<string:ticker>/<string:date_from>/<string:date_to>", methods=["GET"])
def get_span(ticker, date_from, date_to): 
    utc_from = dt.datetime.strptime(date_from, "%Y-%m-%d").timestamp()
    utc_to = dt.datetime.strptime(date_to, "%Y-%m-%d").timestamp()
   # if redis_db.exists(ticker):
    weekly_overview = pd.DataFrame()
    for key in redis_db.keys("GME:*"):
        key = key.decode('utf-8')
        date = key[4:]
        values = redis_db.hmget(key, 'price', 'mentions')
        
        dayly_overview = {
            'Date': date,
            'Price': values[0].decode('utf-8'),
            'Mentions': values[1].decode('utf-8')
        }
        weekly_overview = weekly_overview.append(dayly_overview, ignore_index=True)
        weekly_overview.sort_values('Date')
        
    return weekly_overview.to_json(orient='records')

In [9]:
date_from = '2021-05-24'
date_to = '2021-05-26'
get_span('GME', date_from, date_to)

'[{"Date":"2021-05-26","Mentions":"177","Price":"242"},{"Date":"2021-05-25","Mentions":"2","Price":"209"},{"Date":"2021-05-29","Mentions":"0","Price":"222"},{"Date":"2021-05-24","Mentions":"0","Price":"180"},{"Date":"2021-05-27","Mentions":"10","Price":"254"},{"Date":"2021-05-28","Mentions":"0","Price":"222"}]'

In [10]:
s =redis_db.scard('ticker')
s

0

In [11]:
#weekmap = redis_db.keys(pattern='ticker:*')
#len(weekmap)

In [12]:
#
#redis_db.hset("date", 3, 3)
#redis_db.hset("date", 5, 5)

In [13]:

#mymap = redis_db.keys(pattern='ticker.*')
#len(mymap)

In [ ]:
if __name__ == "__main__":
    api.run(port=5051)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5051/ (Press CTRL+C to quit)
<ipython-input-6-cc0de30aafc9>:10: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  transaction.hmset(ticker +':'+ date, values)#, ex=time_to_expire_s
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/PLTR/2021-05-22/20/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/FORD/2021-05-22/2/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/RIDE/2021-05-22/9/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/SPCE/2021-05-22/21/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/AI/2021-05-22/61/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/TSLA/2021-05-22/580/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/GE/2021-05-22/13/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/GME/2021-05-22/176/0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 11:40:35] "POST /ticker/AAPL/